In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import xrft

In [2]:
dt = np.dtype([('re', np.int16), ('im', np.int16)])

a = np.fromfile(f'out-00.dat', dtype=dt)
			

b = np.zeros(len(a), dtype=np.complex64)
b[:].real = a['re']/(2**15)
b[:].imag = a['im']/(2**15)

sample_rate = int(250e3)
dt = 1/sample_rate

print(f"{len(b)/sample_rate:0.2f}s recorded")



bins = 1024*2

# remove 1 sample at the beginning and end
b = b[sample_rate:-sample_rate]

N = len(b)//bins



time = np.arange(N*bins)*dt
b = b[-N*bins:]

# b = np.split(b,N)
# t = np.split(time, N)



10.00s recorded


In [3]:

da = xr.DataArray(b, dims=["samples"], coords={"time":("samples",time)})
da = da.chunk(bins)



ValueError: unrecognized chunk manager dask - must be one of: []

In [ ]:

da_fft = xrft.fft(da, dim='samples', shift=True, chunks_to_segments=True)

da_fft


In [ ]:
mag_fft = np.abs(da_fft)

#20*np.log10(mag_fft).plot(x="freq_samples", hue="samples_segment", add_legend=False);


In [ ]:
plt.imshow(20*np.log10(mag_fft), aspect='auto')
plt.colorbar();


bins = mag_fft.argmax(dim="freq_samples").to_numpy()

print(bins)

In [ ]:
x= np.angle(da_fft, deg=True)
plt.imshow(x, aspect='auto')
plt.colorbar();

In [ ]:
#get bin with heighest power

assert np.all(np.isclose(bins, bins[0]))


In [ ]:

std = np.std(x[:,bins])
avg = np.mean(x[:, bins])

print(f"std: {std:0.2f}°")
print(f"mean: {avg:0.2f}°")

plt.ylim(avg-10*std, avg+10*std)
plt.plot(x[:, bins]);
